In [1]:
import pandas as pd 
import numpy as np 
import pickle
from tensorflow.keras.models import load_model

In [2]:
## Loading the label encoder, one hot encoding file and standards scaler file
model = load_model("model.h5")

In [3]:
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender = pickle.load(file)
with open("one_hot_encoder_geo.pkl","rb") as file:
    one_hot_encoder_geo = pickle.load(file)
with open("sscaler.pkl","rb") as file:
    sscaler = pickle.load(file) 

In [4]:
df = pd.read_csv("Churn_Modelling.csv")

In [5]:
input_data = pd.DataFrame(df.iloc[1,:])

In [6]:
input_data = input_data.T

In [7]:
input_data["Geography"]

1    Spain
Name: Geography, dtype: object

In [8]:
input_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [9]:
encoded_geo = pd.DataFrame(
    one_hot_encoder_geo.transform(pd.DataFrame(input_data["Geography"])),
    columns = one_hot_encoder_geo.get_feature_names_out(["Geography"]),
    index=input_data.index
)
pd.DataFrame(encoded_geo)


,Geography_France,Geography_Germany,Geography_Spain
1,0.0,0.0,1.0


In [10]:
input_data = pd.concat([input_data,encoded_geo],axis = 1)

In [15]:
input_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
1,2,15647311,Hill,608,Spain,NaN,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [16]:
input_data = input_data.drop(["Geography"],axis=1)

In [19]:
input_data['Gender'] = "Female"

In [20]:
input_data

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [21]:
encoded_gender = label_encoder_gender.transform(input_data["Gender"])
input_data["Gender"] = encoded_gender

In [22]:
input_data

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [23]:
input_data = input_data.drop(["Exited","RowNumber","CustomerId","Surname"],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0


In [24]:
input_scaled = sscaler.transform(input_data)
input_scaled

array([[-0.43240418, -1.08784471,  0.1954709 , -1.39551799,  0.11610125,
        -0.91864709, -1.53629914,  0.97186237,  0.22917551, -1.00080032,
        -0.58330355,  1.75198515]])

In [25]:
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


In [26]:
prediction

array([[0.2929971]], dtype=float32)

In [27]:
prediction_probabilty = prediction[0][0]

In [28]:
prediction_probabilty

0.2929971